# Advanced Pandas: Complete Best Practices Guide

## Comprehensive Coverage of Advanced Pandas Techniques

This notebook demonstrates:
- **Task 1**: Advanced Data Transformation and Reshaping
- **Task 2**: Advanced Merging and Joining
- **Task 3**: Performance Optimization & Memory Management
- **Task 4**: Custom Extensions & Advanced Functionality
- **Task 5**: Real-world Challenge - Retail Analytics

### Key Features:
✓ Clean code architecture with classes
✓ Comprehensive docstrings and comments
✓ Best practices for memory and performance
✓ Production-ready error handling
✓ Real-world data patterns and edge cases

---

## 0. Setup & Configuration

In [60]:
# Standard library imports
import pandas as pd
import numpy as np
import datetime
import uuid
import time
import warnings
import string
import random

# Visualization imports
import matplotlib.pyplot as plt
import seaborn as sns

# Advanced imports
from functools import wraps
from itertools import combinations
from scipy import stats

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)      # Show max 100 rows
pd.set_option('display.float_format', lambda x: f'{x:.2f}')  # Format floats to 2 decimals

print(f"Pandas Version: {pd.__version__}")
print(f"NumPy Version: {np.__version__}")

Pandas Version: 2.2.3
NumPy Version: 2.2.5


## 1. Data Generation Module

### Overview
Generate synthetic datasets for demonstration. We use a custom generator without external dependencies for maximum compatibility.

### Key Points:
- **Reproducibility**: Set random seeds for consistent results
- **Realistic data**: Include missing values, duplicates, outliers
- **Multiple datasets**: Transactions, products, customers, status updates

In [61]:
# ============================================================================
# DATA GENERATOR CLASS - Lightweight alternative to Faker
# ============================================================================

class SimpleDataGenerator:
    """
    Generate realistic fake data without external dependencies.
    Useful for creating sample datasets for pandas demonstrations.
    """
    
    # Predefined lists of realistic names and data
    first_names = ['John', 'Jane', 'Michael', 'Emily', 'David', 'Sarah', 
                   'Robert', 'Jessica', 'James', 'Jennifer', 'Christopher']
    last_names = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia', 
                  'Miller', 'Davis', 'Rodriguez', 'Martinez']
    cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 
              'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'Austin']
    words = ['product', 'item', 'device', 'tool', 'tech', 'gear', 'pro']
    
    @staticmethod
    def name():
        """Generate a random full name"""
        return f"{random.choice(SimpleDataGenerator.first_names)} {random.choice(SimpleDataGenerator.last_names)}"
    
    @staticmethod
    def email():
        """Generate a random email address"""
        name = ''.join(random.choices(string.ascii_lowercase, k=8))
        domain = random.choice(['gmail.com', 'yahoo.com', 'outlook.com', 'email.com'])
        return f"{name}@{domain}"
    
    @staticmethod
    def phone():
        """Generate a random phone number"""
        return f"+1-{random.randint(200, 999)}-{random.randint(200, 999)}-{random.randint(1000, 9999)}"
    
    @staticmethod
    def city():
        """Get a random city"""
        return random.choice(SimpleDataGenerator.cities)
    
    @staticmethod
    def word():
        """Get a random word for product names"""
        return random.choice(SimpleDataGenerator.words)
    
    @staticmethod
    def seed(val):
        """Set seed for reproducibility"""
        random.seed(val)

In [62]:
# ============================================================================
# DATASET GENERATION FUNCTION
# ============================================================================

def generate_datasets(num_customers=1000, num_products=200, num_transactions=50000):
    """
    Generate synthetic datasets for the assignment.
    
    Parameters:
    -----------
    num_customers : int
        Number of customer records to generate (default: 1000)
    num_products : int
        Number of product records to generate (default: 200)
    num_transactions : int
        Number of transaction records to generate (default: 50000)
    
    Returns:
    --------
    tuple : (customers_df, products_df, transactions_df, status_updates_df)
        Four DataFrames containing the generated data
    
    Notes:
    ------
    - Includes realistic missing values (~1% per column)
    - Includes duplicate transactions (~0.5%)
    - Includes negative/invalid amounts for outlier handling
    """
    
    print("="*80)
    print("DATASET GENERATION WITH REALISTIC PATTERNS")
    print("="*80)
    
    # Set seeds for reproducibility - CRITICAL for consistent results
    np.random.seed(42)
    random.seed(42)
    SimpleDataGenerator.seed(42)
    fake = SimpleDataGenerator()
    
    # Define date range
    start_date = datetime.datetime(2020, 1, 1)
    end_date = datetime.datetime(2023, 12, 31)
    days_range = (end_date - start_date).days
    
    # Define business categories
    categories = ['Electronics', 'Clothing', 'Home', 'Food', 'Beauty']
    subcategories = {
        'Electronics': ['Phones', 'Computers', 'Accessories', 'TVs', 'Audio'],
        'Clothing': ['Men', 'Women', 'Children', 'Shoes', 'Accessories'],
        'Home': ['Furniture', 'Kitchen', 'Decor', 'Bedding', 'Bath'],
        'Food': ['Produce', 'Bakery', 'Dairy', 'Meat', 'Beverages'],
        'Beauty': ['Skincare', 'Makeup', 'Haircare', 'Fragrance', 'Bath & Body']
    }
    
    # Geographic regions
    regions = ['North America', 'Europe', 'Asia', 'South America', 'Africa', 'Oceania']
    countries_by_region = {
        'North America': ['USA', 'Canada', 'Mexico'],
        'Europe': ['UK', 'Germany', 'France', 'Italy', 'Spain'],
        'Asia': ['China', 'Japan', 'India', 'South Korea', 'Singapore'],
        'South America': ['Brazil', 'Argentina', 'Colombia', 'Chile', 'Peru'],
        'Africa': ['South Africa', 'Egypt', 'Nigeria', 'Kenya', 'Morocco'],
        'Oceania': ['Australia', 'New Zealand', 'Fiji']
    }
    
    # ========== GENERATE CUSTOMER DATA ==========
    print(f"\n1. Generating {num_customers:,} customer records...")
    customer_ids = [str(uuid.uuid4()) for _ in range(num_customers)]
    customer_data = []
    
    for customer_id in customer_ids:
        # Randomly select region and corresponding country
        region = np.random.choice(regions)
        country = np.random.choice(countries_by_region[region])
        join_date = start_date + datetime.timedelta(days=np.random.randint(0, days_range))
        
        customer_data.append({
            'customer_id': customer_id,
            'name': fake.name(),
            'email': fake.email(),
            'phone': fake.phone(),
            'region': region,
            'country': country,
            'city': fake.city(),
            'join_date': join_date,
            # Customer tier with realistic distribution
            'tier': np.random.choice(['Bronze', 'Silver', 'Gold', 'Platinum'], 
                                    p=[0.5, 0.3, 0.15, 0.05]),
            'is_active': np.random.choice([True, False], p=[0.9, 0.1])
        })
    
    customers_df = pd.DataFrame(customer_data)
    print(f"   ✓ Generated {len(customers_df)} customers")
    
    # ========== GENERATE PRODUCT DATA ==========
    print(f"\n2. Generating {num_products:,} product records...")
    product_ids = [str(uuid.uuid4()) for _ in range(num_products)]
    product_data = []
    
    for product_id in product_ids:
        category = np.random.choice(categories)
        subcategory = np.random.choice(subcategories[category])
        launch_date = start_date + datetime.timedelta(days=np.random.randint(0, days_range))
        
        product_data.append({
            'product_id': product_id,
            'name': fake.word() + ' ' + fake.word().capitalize(),
            'category': category,
            'subcategory': subcategory,
            'price': round(np.random.uniform(10, 1000), 2),
            'cost': round(np.random.uniform(5, 500), 2),
            'weight_kg': round(np.random.uniform(0.1, 20), 2),
            'launch_date': launch_date,
            'is_discontinued': np.random.choice([True, False], p=[0.1, 0.9])
        })
    
    products_df = pd.DataFrame(product_data)
    print(f"   ✓ Generated {len(products_df)} products")
    
    # ========== GENERATE TRANSACTION DATA ==========
    print(f"\n3. Generating {num_transactions:,} transaction records...")
    transaction_data = []
    
    for _ in range(num_transactions):
        transaction_date = start_date + datetime.timedelta(days=np.random.randint(0, days_range))
        customer_id = np.random.choice(customer_ids)
        # Each transaction can have multiple items (1-5)
        num_items = np.random.randint(1, 6)
        
        for _ in range(num_items):
            product_id = np.random.choice(product_ids)
            # Get price from products dataframe
            product_price = products_df.loc[products_df['product_id'] == product_id, 'price'].iloc[0]
            quantity = np.random.randint(1, 5)
            
            # Apply realistic discount distribution
            discount_pct = np.random.choice([0, 0, 0, 0.05, 0.1, 0.15, 0.2], 
                                           p=[0.6, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05])
            price_after_discount = round(product_price * (1 - discount_pct), 2)
            
            transaction_id = str(uuid.uuid4())
            
            transaction_data.append({
                'transaction_id': transaction_id,
                'customer_id': customer_id,
                'product_id': product_id,
                'date': transaction_date,
                'quantity': quantity,
                'unit_price': product_price,
                'discount_pct': discount_pct,
                'price_after_discount': price_after_discount,
                'total_amount': round(quantity * price_after_discount, 2),
                'payment_method': np.random.choice(['Credit Card', 'Debit Card', 'PayPal', 'Cash', 'Bank Transfer']),
                'store_id': np.random.randint(1, 50)
            })
    
    transactions_df = pd.DataFrame(transaction_data)
    print(f"   ✓ Generated {len(transactions_df)} transactions")
    
    # ========== ADD REALISTIC DATA QUALITY ISSUES ==========
    print(f"\n4. Adding realistic data quality issues...")
    
    # Add missing values (~1%)
    transactions_df.loc[np.random.choice(transactions_df.index, size=int(len(transactions_df)*0.01)), 'unit_price'] = np.nan
    transactions_df.loc[np.random.choice(transactions_df.index, size=int(len(transactions_df)*0.01)), 'discount_pct'] = np.nan
    
    # Add anomalies (negative amounts)
    transactions_df.loc[np.random.choice(transactions_df.index, size=int(len(transactions_df)*0.005)), 'total_amount'] = -1
    customers_df.loc[np.random.choice(customers_df.index, size=int(len(customers_df)*0.02)), 'email'] = np.nan
    products_df.loc[np.random.choice(products_df.index, size=int(len(products_df)*0.01)), 'price'] = np.nan
    
    # Add duplicates (~0.5%)
    dupe_indices = np.random.choice(transactions_df.index, size=int(len(transactions_df)*0.005))
    dupes = transactions_df.loc[dupe_indices].copy()
    transactions_df = pd.concat([transactions_df, dupes], ignore_index=True)
    
    print(f"   ✓ Added missing values, anomalies, and duplicates")
    
    # ========== GENERATE STATUS UPDATES ==========
    print(f"\n5. Generating customer status updates...")
    status_updates = []
    for customer_id in customer_ids:
        # Generate 1-5 status updates per customer
        num_updates = np.random.randint(1, 6)
        for _ in range(num_updates):
            update_date = start_date + datetime.timedelta(days=np.random.randint(0, days_range))
            status_updates.append({
                'customer_id': customer_id,
                'update_date': update_date,
                'tier': np.random.choice(['Bronze', 'Silver', 'Gold', 'Platinum']),
                'lifetime_value': round(np.random.uniform(0, 10000), 2),
                'credit_score': np.random.randint(300, 851)
            })
    
    status_updates_df = pd.DataFrame(status_updates)
    status_updates_df.sort_values('update_date', inplace=True)
    print(f"   ✓ Generated {len(status_updates_df)} status updates")
    
    # ========== SUMMARY ==========
    print(f"\n" + "-"*80)
    print("DATASET SUMMARY")
    print("-"*80)
    print(f"\nCustomers: {len(customers_df)} records, {customers_df.shape[1]} columns")
    print(f"Products: {len(products_df)} records, {products_df.shape[1]} columns")
    print(f"Transactions: {len(transactions_df)} records, {transactions_df.shape[1]} columns")
    print(f"Status Updates: {len(status_updates_df)} records, {status_updates_df.shape[1]} columns")
    
    return customers_df, products_df, transactions_df, status_updates_df


# Generate datasets
print("\nGenerating datasets...\n")
customers_df, products_df, transactions_df, status_updates_df = generate_datasets(
    num_customers=500,      # Reduced for faster execution
    num_products=100,
    num_transactions=10000
)


Generating datasets...

DATASET GENERATION WITH REALISTIC PATTERNS

1. Generating 500 customer records...
   ✓ Generated 500 customers

2. Generating 100 product records...
   ✓ Generated 100 products

3. Generating 10,000 transaction records...
   ✓ Generated 30170 transactions

4. Adding realistic data quality issues...
   ✓ Added missing values, anomalies, and duplicates

5. Generating customer status updates...
   ✓ Generated 1464 status updates

--------------------------------------------------------------------------------
DATASET SUMMARY
--------------------------------------------------------------------------------

Customers: 500 records, 10 columns
Products: 100 records, 9 columns
Transactions: 30320 records, 11 columns
Status Updates: 1464 records, 5 columns


## 2. TASK 1: Advanced Data Transformation & Reshaping

### Topics Covered:
1. **Pivot/Melt Operations** - Converting between long and wide formats
2. **Multi-level Indexing** - Creating hierarchical indices
3. **Advanced GroupBy** - Using transform, agg, and custom functions

### Best Practices:
- Use `pivot_table()` for flexible reshaping
- Use `melt()` to unpivot data
- Use `groupby().transform()` for broadcasting aggregations back to original shape
- Use `.xs()` for cross-section extraction from multi-index

In [63]:
# ============================================================================
# TASK 1: ADVANCED DATA TRANSFORMATION AND RESHAPING
# ============================================================================

print("\n" + "█"*80)
print("█ TASK 1: ADVANCED DATA TRANSFORMATION AND RESHAPING")
print("█"*80)

class Task1_DataTransformation:
    """
    Demonstrates advanced pandas data transformation techniques.
    
    Methods:
    - pivot_and_melt_operations: Convert between long/wide formats
    - multilevel_indexing: Create and manipulate hierarchical indices
    - advanced_groupby_operations: Advanced aggregation and filtering
    """
    
    def __init__(self, transactions_df, products_df, customers_df):
        """
        Initialize Task 1 with data references.
        
        Parameters:
        -----------
        transactions_df : DataFrame
            Transaction records
        products_df : DataFrame
            Product master data
        customers_df : DataFrame
            Customer master data
        """
        # Create copies to avoid modifying original data
        self.transactions = transactions_df.copy()
        self.products = products_df.copy()
        self.customers = customers_df.copy()
    
    def pivot_and_melt_operations(self):
        """
        Demonstrate pivot (long to wide) and melt (wide to long) operations.
        
        Key Concepts:
        - pivot_table(): More flexible, allows aggregation
        - melt(): Convert wide format back to long
        - margins: Add row/column totals
        """
        print("\n" + "="*80)
        print("1.1: PIVOT AND MELT OPERATIONS")
        print("="*80)
        
        # Merge to get category information
        df = self.transactions.merge(
            self.products[['product_id', 'category']], 
            on='product_id', 
            how='left'
        )
        
        # Create year-month period for analysis
        df['year_month'] = df['date'].dt.to_period('M')
        
        # ===== PIVOT: Long to Wide Format =====
        print("\n1. PIVOT: Converting to Wide Format")
        print("-" * 40)
        
        # Create pivot table with monthly sales by category
        pivot_wide = df.pivot_table(
            index='year_month',           # Rows
            columns='category',           # Columns
            values='total_amount',        # Values to aggregate
            aggfunc='sum',                # Aggregation function
            fill_value=0                  # Fill missing values with 0
        )
        
        print(f"Pivot Shape: {pivot_wide.shape}")
        print("\nFirst 5 rows:")
        print(pivot_wide.head())
        
        # ===== MELT: Wide to Long Format =====
        print("\n2. MELT: Converting back to Long Format")
        print("-" * 40)
        
        # Reset index to make year_month a column
        pivot_reset = pivot_wide.reset_index()
        
        # Melt back to long format
        melted = pivot_reset.melt(
            id_vars=['year_month'],         # Columns to keep as-is
            var_name='category',            # Name for pivoted columns
            value_name='total_sales'        # Name for values
        )
        
        print(f"Melted Shape: {melted.shape}")
        print("\nFirst 10 rows:")
        print(melted.head(10))
        
        # ===== PIVOT WITH MARGINS (Totals) =====
        print("\n3. PIVOT WITH MARGINS: Adding Row and Column Totals")
        print("-" * 40)
        
        # Add month column for granular analysis
        df_temp = df.copy()
        df_temp['month'] = df_temp['date'].dt.month
        
        # Pivot with margins=True adds totals
        pivot_with_totals = df_temp.pivot_table(
            index='category',
            columns='month',
            values='total_amount',
            aggfunc='sum',
            margins=True,                   # Add totals
            margins_name='Total'
        )
        
        print(f"Shape with Margins: {pivot_with_totals.shape}")
        print("\nPivot with Totals:")
        print(pivot_with_totals)
        
        return pivot_wide, melted, pivot_with_totals
    
    def multilevel_indexing(self):
        """
        Demonstrate hierarchical (multi-level) indexing.
        
        Key Concepts:
        - set_index(): Create multi-level index
        - xs(): Cross-section extraction
        - unstack()/stack(): Reshape hierarchical data
        """
        print("\n" + "="*80)
        print("1.2: MULTI-LEVEL INDEXING")
        print("="*80)
        
        # Merge necessary data
        df = self.transactions.merge(
            self.products[['product_id', 'category']], 
            on='product_id', 
            how='left'
        ).merge(
            self.customers[['customer_id', 'region']], 
            on='customer_id', 
            how='left'
        )
        
        # Create date column for indexing
        df['date_only'] = df['date'].dt.date
        
        # ===== CREATE HIERARCHICAL INDEX =====
        print("\n1. Creating Hierarchical Index")
        print("-" * 40)
        
        # Create 3-level index: region > category > date
        hierarchical = df.set_index(['region', 'category', 'date_only']).sort_index()
        
        print(f"Shape: {hierarchical.shape}")
        print(f"Index Levels: {hierarchical.index.names}")
        print(f"Number of Levels: {hierarchical.index.nlevels}")
        print("\nSample data:")
        print(hierarchical.head(10))
        
        # ===== CROSS-SECTION (XS) OPERATIONS =====
        print("\n2. Cross-Section (xs) Operations")
        print("-" * 40)
        
        # Extract data for specific region
        regions_available = hierarchical.index.get_level_values(0).unique()
        first_region = regions_available[0]
        
        subset = hierarchical.xs(first_region, level='region')
        print(f"\nData for {first_region} region: {subset.shape[0]} rows")
        print(subset.head())
        
        # ===== UNSTACK/RESTACK OPERATIONS =====
        print("\n3. Unstack and Restack Operations")
        print("-" * 40)
        
        # First aggregate to remove duplicates
        hierarchical_agg = hierarchical['total_amount'].groupby(level=[0, 1, 2]).sum()
        
        # Unstack: Convert index level to columns
        unstacked = hierarchical_agg.unstack(level=0, fill_value=0)
        print(f"\nUnstacked Shape: {unstacked.shape}")
        print("(Regions now as columns)")
        print(unstacked.head())
        
        # Restack: Convert columns back to index
        restacked = unstacked.stack()
        print(f"\nRestacked Shape: {restacked.shape}")
        print("(Back to hierarchical structure)")
        print(restacked.head(10))
        
        return hierarchical, unstacked, restacked
    
    def advanced_groupby_operations(self):
        """
        Demonstrate advanced groupby operations.
        
        Key Concepts:
        - transform(): Apply function to each group and broadcast result
        - agg(): Apply multiple aggregation functions
        - filter(): Select groups meeting criteria
        - Custom aggregation functions
        """
        print("\n" + "="*80)
        print("1.3: ADVANCED GROUPBY OPERATIONS")
        print("="*80)
        
        df = self.transactions.merge(
            self.products[['product_id', 'category']], 
            on='product_id', 
            how='left'
        )
        
        # ===== TRANSFORM: Group and Broadcast =====
        print("\n1. Using Transform to Normalize Values")
        print("-" * 40)
        
        # Transform normalizes each value relative to its group
        df['category_normalized'] = df.groupby('category')['total_amount'].transform(
            lambda x: (x - x.mean()) / x.std()  # Z-score normalization
        )
        
        print("Normalized values (z-score per category):")
        print(df[['category', 'total_amount', 'category_normalized']].head(10))
        
        # ===== MULTIPLE AGGREGATIONS =====
        print("\n2. Applying Multiple Aggregation Functions")
        print("-" * 40)
        
        # Apply different functions to different columns
        agg_result = df.groupby('category').agg({
            'total_amount': ['sum', 'mean', 'std', 'min', 'max', 'count'],
            'quantity': ['mean', 'median'],
            'discount_pct': 'mean'
        })
        
        print(f"\nAggregation Result Shape: {agg_result.shape}")
        print(agg_result)
        
        # ===== CUSTOM AGGREGATION FUNCTION =====
        print("\n3. Custom Aggregation Function")
        print("-" * 40)
        
        def revenue_variance_ratio(x):
            """Calculate variance to mean ratio (coefficient of variation)"""
            if len(x) > 0 and x.mean() != 0:
                return x.var() / x.mean()
            return 0
        
        # Apply custom function
        custom_agg = df.groupby('category')['total_amount'].agg([
            ('Total Revenue', 'sum'),
            ('Mean Sale', 'mean'),
            ('Var-to-Mean Ratio', revenue_variance_ratio)
        ])
        
        print("\nCustom Aggregation Results:")
        print(custom_agg)
        
        # ===== FILTER: Select Groups Meeting Criteria =====
        print("\n4. Filtering Groups by Criteria")
        print("-" * 40)
        
        # Calculate category totals
        category_totals = df.groupby('category')['total_amount'].sum()
        median_sales = category_totals.median()
        
        # Filter: Keep only rows from high-sales categories
        high_sales = df.groupby('category').filter(
            lambda x: x['total_amount'].sum() > median_sales
        )
        
        print(f"\nCategories with sales > median (${median_sales:,.2f}):")
        print(f"  Filtered records: {len(high_sales)}")
        print(f"  Categories: {high_sales['category'].unique().tolist()}")
        
        return agg_result, custom_agg, high_sales


# Execute Task 1
task1 = Task1_DataTransformation(transactions_df, products_df, customers_df)
pivot_wide, melted, pivot_totals = task1.pivot_and_melt_operations()
hierarchical, unstacked, restacked = task1.multilevel_indexing()
agg_result, custom_agg, high_sales = task1.advanced_groupby_operations()

print("\n✓ Task 1 Completed Successfully!")


████████████████████████████████████████████████████████████████████████████████
█ TASK 1: ADVANCED DATA TRANSFORMATION AND RESHAPING
████████████████████████████████████████████████████████████████████████████████

1.1: PIVOT AND MELT OPERATIONS

1. PIVOT: Converting to Wide Format
----------------------------------------
Pivot Shape: (48, 5)

First 5 rows:
category      Beauty  Clothing  Electronics      Food      Home
year_month                                                     
2020-01    191416.13  79410.26    207781.54 216662.44 123052.83
2020-02    143038.66  70213.56    147232.19 176362.88 106198.56
2020-03    160557.49  96129.90    202878.24 218191.37 148982.16
2020-04    168171.00  78350.09    185055.15 238661.47 125440.87
2020-05    135484.29  55955.21    180298.41 257277.22 124748.90

2. MELT: Converting back to Long Format
----------------------------------------
Melted Shape: (240, 3)

First 10 rows:
  year_month category  total_sales
0    2020-01   Beauty    191416.13

## 3. TASK 2: Advanced Merging & Joining

### Topics Covered:
1. **Complex Joins** - Three-way joins, self-joins, different join types
2. **Duplicate Handling** - Identify and resolve duplicates
3. **Time-Based Joins** - AsOf joins, time-window joins

### Best Practices:
- Understand join semantics (inner, left, right, outer)
- Check for duplicates before merging
- Handle column name conflicts with suffixes
- Validate merge integrity post-operation

In [64]:
# ============================================================================
# TASK 2: ADVANCED MERGING AND JOINING
# ============================================================================

print("\n" + "█"*80)
print("█ TASK 2: ADVANCED MERGING AND JOINING")
print("█"*80)

class Task2_MergingAndJoining:
    """
    Demonstrates advanced pandas joining and merging techniques.
    
    Methods:
    - complex_joins: Multiple joins, self-joins, join types comparison
    - handling_duplicates: Identify and resolve duplicate keys
    - time_based_joins: AsOf joins and time-window joins
    """
    
    def __init__(self, transactions_df, products_df, customers_df, status_updates_df):
        """Initialize with all necessary dataframes"""
        self.transactions = transactions_df.copy()
        self.products = products_df.copy()
        self.customers = customers_df.copy()
        self.status_updates = status_updates_df.copy()
    
    def complex_joins(self):
        """
        Demonstrate complex join operations.
        
        Includes:
        - Three-way joins combining multiple datasets
        - Self-joins for finding relationships
        - Comparison of different join types
        """
        print("\n" + "="*80)
        print("2.1: COMPLEX JOINS")
        print("="*80)
        
        # ===== THREE-WAY JOIN =====
        print("\n1. Three-Way Join (Transactions + Customers + Products)")
        print("-" * 40)
        
        # Chain multiple merges: transactions > customers > products
        three_way = (
            self.transactions
            .merge(self.customers[['customer_id', 'region', 'tier']], 
                  on='customer_id', how='left')
            .merge(self.products[['product_id', 'category', 'price']], 
                  on='product_id', how='left', suffixes=('_trans', '_prod'))
        )
        
        print(f"Three-way join shape: {three_way.shape}")
        print(f"Columns: {list(three_way.columns)[:15]}...")  # Show first 15
        print("\nSample data:")
        print(three_way[['customer_id', 'region', 'tier', 'product_id', 'category']].head(10))
        
        # ===== SELF-JOIN =====
        print("\n2. Self-Join (Finding Related Records)")
        print("-" * 40)
        
        # Self-join to find customers in same region
        customers_subset = self.customers[['customer_id', 'region', 'tier']].head(100)
        
        self_join = (
            customers_subset
            .merge(customers_subset, on='region', suffixes=('_cust1', '_cust2'))
            .query('customer_id_cust1 != customer_id_cust2')  # Remove same customer
        )
        
        print(f"Self-join result shape: {self_join.shape}")
        print("\nSample (customers in same region):")
        print(self_join[['customer_id_cust1', 'customer_id_cust2', 'region']].head(10))
        
        # ===== COMPARE JOIN TYPES =====
        print("\n3. Comparing Different Join Types")
        print("-" * 40)
        
        # Use small subset for demonstration
        small_trans = self.transactions.head(50)
        small_products = self.products.head(40)
        
        join_comparison = {}
        for how in ['inner', 'left', 'right', 'outer']:
            joined = small_trans.merge(
                small_products[['product_id', 'category']],
                on='product_id',
                how=how
            )
            join_comparison[how] = len(joined)
            print(f"  {how.upper():6s}: {len(joined):4d} rows")
        
        return three_way, self_join, join_comparison
    
    def handling_duplicates_and_conflicts(self):
        """
        Identify and handle duplicates and conflicts in merging.
        
        Includes:
        - Duplicate detection
        - Merge validation
        - Conflict resolution
        """
        print("\n" + "="*80)
        print("2.2: HANDLING DUPLICATES AND CONFLICTS")
        print("="*80)
        
        # ===== IDENTIFY DUPLICATES =====
        print("\n1. Identifying Duplicate Records")
        print("-" * 40)
        
        # Check for duplicate transaction IDs
        dup_mask = self.transactions.duplicated(subset=['transaction_id'], keep=False)
        num_dups = dup_mask.sum()
        
        print(f"Duplicate transaction_ids: {num_dups}")
        if num_dups > 0:
            print("\nSample duplicates:")
            print(self.transactions[dup_mask][['transaction_id', 'customer_id', 'total_amount']].head(10))
        
        # ===== MERGE VALIDATION =====
        print("\n2. Merge Validation Function")
        print("-" * 40)
        
        def validate_merge(left_df, right_df, on_col, merge_type='left'):
            """
            Validate merge integrity.
            
            Returns:
            --------
            dict : Validation metrics
            """
            left_count = len(left_df)
            merged = left_df.merge(right_df, on=on_col, how=merge_type)
            merged_count = len(merged)
            
            return {
                'left_rows': left_count,
                'merged_rows': merged_count,
                'rows_lost': left_count - merged_count,
                'valid': left_count == merged_count
            }
        
        # Test validation
        validation = validate_merge(
            self.transactions.head(1000),
            self.products[['product_id', 'category']],
            'product_id'
        )
        
        print(f"\nValidation Results:")
        for key, value in validation.items():
            print(f"  {key}: {value}")
        
        # ===== CONFLICT RESOLUTION =====
        print("\n3. Handling Column Name Conflicts")
        print("-" * 40)
        
        # Create subsets with overlapping column names
        cust_subset = self.customers[['customer_id', 'tier']].copy().rename(columns={'tier': 'customer_tier'})
        status_subset = self.status_updates[['customer_id', 'tier']].copy().rename(columns={'tier': 'status_tier'})
        
        # Merge with explicit suffixes
        merged_conflict = cust_subset.merge(status_subset, on='customer_id', how='left')
        
        print(f"Merged shape: {merged_conflict.shape}")
        print(f"Columns: {list(merged_conflict.columns)}")
        print("\nSample data:")
        print(merged_conflict.head(10))
        
        return num_dups, validation, merged_conflict
    
    def time_based_joins(self):
        """
        Perform time-based joins.
        
        Includes:
        - AsOf joins for matching by nearest timestamp
        - Time-window joins for events within time range
        """
        print("\n" + "="*80)
        print("2.3: TIME-BASED JOINS")
        print("="*80)
        
        # Sort by time (required for some join operations)
        trans_sorted = self.transactions.sort_values('date').copy()
        status_sorted = self.status_updates.sort_values('update_date').copy()
        
        # ===== ASOF JOIN =====
        print("\n1. AsOf Join - Match by Nearest Previous Timestamp")
        print("-" * 40)
        
        # Rename columns for clarity
        trans_for_asof = trans_sorted[['customer_id', 'date', 'total_amount']].head(100).copy()
        status_for_asof = status_sorted[['customer_id', 'update_date', 'tier']].copy()
        
        # Perform asof join
        asof_result = pd.merge_asof(
            trans_for_asof.rename(columns={'date': 'transaction_date'}),
            status_for_asof,
            left_on='transaction_date',      # Left join key
            right_on='update_date',          # Right join key
            by='customer_id',                # Join by customer
            direction='backward'             # Match previous status
        )
        
        print(f"AsOf join result shape: {asof_result.shape}")
        print("\nSample data (transactions with matched status):")
        print(asof_result[['customer_id', 'transaction_date', 'update_date', 'tier']].head(10))
        
        # ===== TIME WINDOW JOIN =====
        print("\n2. Time Window Join - Match Within 30 Days")
        print("-" * 40)
        
        # Create sample for faster processing
        sample_trans = trans_sorted.head(50)[['customer_id', 'date', 'total_amount']].copy()
        sample_status = status_sorted[status_sorted['customer_id'].isin(sample_trans['customer_id'].unique())].copy()
        
        window_joins = []
        for _, trans_row in sample_trans.iterrows():
            cust_id = trans_row['customer_id']
            trans_date = trans_row['date']
            
            # Find status updates within 30 days before transaction
            matching = sample_status[
                (sample_status['customer_id'] == cust_id) &
                (sample_status['update_date'] <= trans_date) &
                (sample_status['update_date'] >= trans_date - pd.Timedelta(days=30))
            ]
            
            if len(matching) > 0:
                # Use most recent update
                most_recent = matching.sort_values('update_date').iloc[-1]
                window_joins.append({
                    'customer_id': cust_id,
                    'transaction_date': trans_date,
                    'status_date': most_recent['update_date'],
                    'days_before': (trans_date - most_recent['update_date']).days
                })
        
        window_result = pd.DataFrame(window_joins)
        print(f"\nWindow join result shape: {window_result.shape}")
        print("\nSample data (within 30-day window):")
        print(window_result.head(10))
        
        return asof_result, window_result


# Execute Task 2
task2 = Task2_MergingAndJoining(transactions_df, products_df, customers_df, status_updates_df)
three_way, self_join, join_types = task2.complex_joins()
num_dups, validation, merged_conflict = task2.handling_duplicates_and_conflicts()
asof_result, window_result = task2.time_based_joins()

print("\n✓ Task 2 Completed Successfully!")


████████████████████████████████████████████████████████████████████████████████
█ TASK 2: ADVANCED MERGING AND JOINING
████████████████████████████████████████████████████████████████████████████████

2.1: COMPLEX JOINS

1. Three-Way Join (Transactions + Customers + Products)
----------------------------------------
Three-way join shape: (30320, 15)
Columns: ['transaction_id', 'customer_id', 'product_id', 'date', 'quantity', 'unit_price', 'discount_pct', 'price_after_discount', 'total_amount', 'payment_method', 'store_id', 'region', 'tier', 'category', 'price']...

Sample data:
                            customer_id         region    tier  \
0  7643eb8a-faab-4ac0-8cfb-4309b1befce2  South America  Bronze   
1  7643eb8a-faab-4ac0-8cfb-4309b1befce2  South America  Bronze   
2  2b701961-a805-4520-911e-65a5d7505155  North America  Bronze   
3  66751b17-40b4-41f2-82f9-008dd3cc8e1e           Asia    Gold   
4  66751b17-40b4-41f2-82f9-008dd3cc8e1e           Asia    Gold   
5  77abe375-c402-

## 4. TASK 3: Performance Optimization & Memory Management

### Topics Covered:
1. **Memory Optimization** - Dtype conversion, categoricals
2. **Computational Efficiency** - Vectorization vs iterrows
3. **Chunking Strategy** - Processing large datasets

### Best Practices:
- Always use vectorized operations over loops
- Use `np.where()` for conditional operations
- Downcast numeric types when possible
- Process data in chunks for memory efficiency

In [65]:
# ============================================================================
# TASK 3: PERFORMANCE OPTIMIZATION
# ============================================================================

print("\n" + "█"*80)
print("█ TASK 3: PERFORMANCE OPTIMIZATION & MEMORY MANAGEMENT")
print("█"*80)

class Task3_PerformanceOptimization:
    """
    Demonstrates performance and memory optimization techniques.
    
    Methods:
    - memory_usage_optimization: Reduce memory footprint
    - computational_efficiency: Vectorization vs iterrows
    - chunking_strategy: Large dataset processing
    """
    
    def __init__(self, transactions_df, products_df, customers_df):
        """Initialize with dataframes"""
        self.transactions = transactions_df.copy()
        self.products = products_df.copy()
        self.customers = customers_df.copy()
    
    def memory_usage_optimization(self):
        """
        Analyze and optimize memory usage.
        
        Strategies:
        - Convert object columns to category
        - Downcast numeric types (int64→int32, float64→float32)
        - Drop unnecessary columns
        """
        print("\n" + "="*80)
        print("3.1: MEMORY USAGE OPTIMIZATION")
        print("="*80)
        
        df_before = self.transactions.copy()
        memory_before = df_before.memory_usage(deep=True).sum() / 1024**2
        
        print(f"\n1. BEFORE OPTIMIZATION")
        print("-" * 40)
        print(f"Total memory: {memory_before:.2f} MB")
        print(f"\nMemory by column (KB):")
        print(df_before.memory_usage(deep=True) / 1024)
        
        # ===== OPTIMIZE DTYPES =====
        print(f"\n2. OPTIMIZING DATA TYPES")
        print("-" * 40)
        
        df_after = self.transactions.copy()
        
        # Convert object columns to category (string columns with low cardinality)
        categorical_cols = ['payment_method']
        for col in categorical_cols:
            if col in df_after.columns:
                df_after[col] = df_after[col].astype('category')
                print(f"  - {col}: converted to category")
        
        # Downcast numeric columns to smaller types
        # int64 → int32 or int16 if values fit
        if 'quantity' in df_after.columns:
            df_after['quantity'] = pd.to_numeric(df_after['quantity'], downcast='integer')
            print(f"  - quantity: downcasted to {df_after['quantity'].dtype}")
        
        # float64 → float32 (acceptable precision loss for many applications)
        float_cols = df_after.select_dtypes(include=['float64']).columns
        for col in float_cols:
            df_after[col] = df_after[col].astype('float32')
        print(f"  - Float columns: converted float64 → float32")
        
        memory_after = df_after.memory_usage(deep=True).sum() / 1024**2
        reduction = ((memory_before - memory_after) / memory_before) * 100
        
        print(f"\n3. AFTER OPTIMIZATION")
        print("-" * 40)
        print(f"Total memory: {memory_after:.2f} MB")
        print(f"\n✓ Memory reduction: {reduction:.1f}%")
        print(f"  Savings: {memory_before - memory_after:.2f} MB")
        
        return memory_before, memory_after, df_after
    
    def computational_efficiency(self):
        """
        Compare efficiency of different operations.
        
        Demonstrates:
        - Vectorized operations (fastest)
        - np.where() for conditionals
        - Iterrows (slow - DON'T USE)
        """
        print("\n" + "="*80)
        print("3.2: COMPUTATIONAL EFFICIENCY")
        print("="*80)
        
        df = self.transactions.head(10000).copy()
        
        # ===== VECTORIZED VS ITERROWS =====
        print("\n1. Vectorized vs Iterrows")
        print("-" * 40)
        
        # VECTORIZED (FAST) - Use this!
        start = time.time()
        result_vec = df['total_amount'] * 2
        vec_time = time.time() - start
        
        # ITERROWS (SLOW) - Never use this for large datasets
        start = time.time()
        result_iter = []
        for idx, row in df.iterrows():
            result_iter.append(row['total_amount'] * 2)
        iter_time = time.time() - start
        
        print(f"Vectorized: {vec_time*1000:.4f} ms")
        print(f"Iterrows:   {iter_time*1000:.4f} ms")
        print(f"\n✓ Speedup: {iter_time/vec_time:.0f}x faster with vectorization!")
        
        # ===== CONDITIONAL OPERATIONS =====
        print("\n2. Efficient Conditional Operations")
        print("-" * 40)
        
        # Method 1: np.where (very fast)
        start = time.time()
        result_where = np.where(
            df['discount_pct'] > 0.1, 'High',
            np.where(df['discount_pct'] > 0.05, 'Medium', 'Low')
        )
        where_time = time.time() - start
        
        # Method 2: pd.cut (also fast)
        start = time.time()
        result_cut = pd.cut(
            df['total_amount'],
            bins=[0, 50, 100, 500, float('inf')],
            labels=['Budget', 'Standard', 'Premium', 'Luxury']
        )
        cut_time = time.time() - start
        
        print(f"np.where():  {where_time*1000:.4f} ms")
        print(f"pd.cut():    {cut_time*1000:.4f} ms")
        
        return vec_time, iter_time, where_time, cut_time
    
    def chunking_strategy(self):
        """
        Demonstrate chunking for large dataset processing.
        
        Useful for:
        - Processing data larger than RAM
        - Batch operations
        - Parallel processing
        """
        print("\n" + "="*80)
        print("3.3: CHUNKING STRATEGY FOR LARGE DATASETS")
        print("="*80)
        
        def chunk_dataframe(df, chunk_size=5000):
            """
            Yield successive chunks from DataFrame.
            
            Useful as generator to avoid loading full dataset at once.
            """
            for i in range(0, len(df), chunk_size):
                yield df.iloc[i:i + chunk_size]
        
        print(f"\nProcessing {len(self.transactions):,} rows in chunks of 5000...\n")
        
        chunk_results = []
        total_rows = 0
        
        for chunk_num, chunk in enumerate(chunk_dataframe(self.transactions, chunk_size=5000), 1):
            # Process each chunk
            chunk_summary = {
                'chunk_num': chunk_num,
                'rows': len(chunk),
                'total_sales': chunk['total_amount'].sum(),
                'avg_sale': chunk['total_amount'].mean(),
                'unique_customers': chunk['customer_id'].nunique()
            }
            chunk_results.append(chunk_summary)
            total_rows += len(chunk)
            
            print(f"  Chunk {chunk_num}: {len(chunk):6d} rows | "
                  f"Sales: ${chunk['total_amount'].sum():12,.2f} | "
                  f"Customers: {chunk['customer_id'].nunique():5d}")
        
        chunk_df = pd.DataFrame(chunk_results)
        
        print(f"\nTotal rows processed: {total_rows:,}")
        print(f"Average chunk size: {chunk_df['rows'].mean():.0f}")
        
        return chunk_results


# Execute Task 3
task3 = Task3_PerformanceOptimization(transactions_df, products_df, customers_df)
mem_before, mem_after, df_optimized = task3.memory_usage_optimization()
vec_time, iter_time, where_time, cut_time = task3.computational_efficiency()
chunks = task3.chunking_strategy()

print("\n✓ Task 3 Completed Successfully!")


████████████████████████████████████████████████████████████████████████████████
█ TASK 3: PERFORMANCE OPTIMIZATION & MEMORY MANAGEMENT
████████████████████████████████████████████████████████████████████████████████

3.1: MEMORY USAGE OPTIMIZATION

1. BEFORE OPTIMIZATION
----------------------------------------
Total memory: 14.37 MB

Memory by column (KB):
Index                     0.13
transaction_id         2753.67
customer_id            3701.17
product_id             3701.17
date                    236.88
quantity                236.88
unit_price              236.88
discount_pct            236.88
price_after_discount    236.88
total_amount            236.88
payment_method         2895.91
store_id                236.88
dtype: float64

2. OPTIMIZING DATA TYPES
----------------------------------------
  - payment_method: converted to category
  - quantity: downcasted to int8
  - Float columns: converted float64 → float32

3. AFTER OPTIMIZATION
---------------------------------------

## 5. TASK 4: Custom Extensions & Advanced Functionality

### Topics Covered:
1. **Custom Accessors** - Extend DataFrame with custom methods
2. **Method Chaining** - Build reusable data pipelines
3. **Advanced Analysis** - Complex pandas operations

### Best Practices:
- Register custom accessors for domain-specific operations
- Use method chaining for readable pipelines
- Implement docstrings for all custom methods

In [66]:
# ============================================================================
# TASK 4: CUSTOM EXTENSIONS AND ADVANCED FUNCTIONALITY
# ============================================================================

print("\n" + "█"*80)
print("█ TASK 4: CUSTOM EXTENSIONS & ADVANCED FUNCTIONALITY")
print("█"*80)

# Register custom pandas accessor for sales data
@pd.api.extensions.register_dataframe_accessor("sales")
class SalesAccessor:
    """
    Custom accessor for sales-specific operations.
    
    Usage:
        df.sales.total_revenue()
        df.sales.discount_analysis()
        etc.
    """
    
    def __init__(self, pandas_obj):
        """Store reference to parent dataframe"""
        self._obj = pandas_obj
    
    def total_revenue(self):
        """Calculate total revenue"""
        return self._obj['total_amount'].sum()
    
    def avg_transaction(self):
        """Calculate average transaction value"""
        return self._obj['total_amount'].mean()
    
    def revenue_by_category(self):
        """Get revenue breakdown by category"""
        if 'category' in self._obj.columns:
            return self._obj.groupby('category')['total_amount'].sum().sort_values(ascending=False)
        return None
    
    def discount_analysis(self):
        """Analyze impact of discounts on revenue"""
        if 'discount_pct' in self._obj.columns and 'total_amount' in self._obj.columns:
            discounted = self._obj[self._obj['discount_pct'] > 0]
            return {
                'pct_discounted': (len(discounted) / len(self._obj)) * 100,
                'avg_discount_pct': discounted['discount_pct'].mean() * 100,
                'total_revenue_discounted': discounted['total_amount'].sum()
            }
        return None


# Register quality accessor
@pd.api.extensions.register_dataframe_accessor("quality")
class QualityAccessor:
    """
    Custom accessor for data quality metrics.
    
    Usage:
        df.quality.completeness()
        df.quality.duplicate_ratio
        etc.
    """
    
    def __init__(self, pandas_obj):
        """Store reference to parent dataframe"""
        self._obj = pandas_obj
    
    def completeness(self):
        """Calculate data completeness percentage"""
        total_cells = self._obj.size
        null_cells = self._obj.isnull().sum().sum()
        return ((total_cells - null_cells) / total_cells) * 100
    
    @property
    def duplicate_ratio(self):
        """Calculate duplicate row percentage"""
        total_rows = len(self._obj)
        duplicate_rows = self._obj.duplicated().sum()
        return (duplicate_rows / total_rows) * 100 if total_rows > 0 else 0
    
    @property
    def missing_summary(self):
        """Get missing value summary by column"""
        return (self._obj.isnull().sum() / len(self._obj) * 100).round(2)


class Task4_CustomExtensions:
    """
    Demonstrates custom accessors and method chaining.
    """
    
    def __init__(self, transactions_df, products_df, customers_df):
        """Initialize with dataframes"""
        self.transactions = transactions_df.copy()
        self.products = products_df.copy()
        self.customers = customers_df.copy()
    
    def custom_accessors_demo(self):
        """
        Demonstrate custom pandas accessors.
        
        Custom accessors allow extending DataFrame with domain-specific methods.
        """
        print("\n" + "="*80)
        print("4.1: CUSTOM ACCESSORS")
        print("="*80)
        
        # Merge data
        df = self.transactions.merge(
            self.products[['product_id', 'category']],
            on='product_id',
            how='left'
        )
        
        # ===== SALES ACCESSOR =====
        print("\n1. Sales Accessor Methods")
        print("-" * 40)
        
        print(f"Total Revenue: ${df.sales.total_revenue():,.2f}")
        print(f"Avg Transaction: ${df.sales.avg_transaction():.2f}")
        
        print("\nRevenue by Category:")
        revenue_by_cat = df.sales.revenue_by_category()
        for category, revenue in revenue_by_cat.items():
            pct = (revenue / df.sales.total_revenue()) * 100
            print(f"  {category:15s}: ${revenue:12,.2f} ({pct:5.1f}%)")
        
        print("\nDiscount Analysis:")
        discount_info = df.sales.discount_analysis()
        if discount_info:
            print(f"  % Transactions Discounted: {discount_info['pct_discounted']:.1f}%")
            print(f"  Average Discount: {discount_info['avg_discount_pct']:.2f}%")
            print(f"  Revenue (Discounted): ${discount_info['total_revenue_discounted']:,.2f}")
        
        # ===== QUALITY ACCESSOR =====
        print("\n2. Quality Accessor Methods")
        print("-" * 40)
        
        print(f"Data Completeness: {df.quality.completeness():.1f}%")
        print(f"Duplicate Ratio: {df.quality.duplicate_ratio:.2f}%")
        
        print("\nMissing Values by Column:")
        missing = df.quality.missing_summary
        for col, pct in missing[missing > 0].items():
            print(f"  {col}: {pct:.2f}%")
        
        return df
    
    def method_chaining_pipeline(self):
        """
        Create reusable data processing pipeline using method chaining.
        
        Method chaining improves readability and enables functional programming style.
        """
        print("\n" + "="*80)
        print("4.2: METHOD CHAINING PIPELINES")
        print("="*80)
        
        print("\nBuilding data processing pipeline...")
        print("-" * 40)
        
        # Create pipeline using method chaining
        result = (
            self.transactions
            .copy()
            # Step 1: Filter valid transactions
            .query('total_amount > 0')
            # Step 2: Merge with product info
            .merge(self.products[['product_id', 'category']], on='product_id', how='left')
            # Step 3: Merge with customer info
            .merge(self.customers[['customer_id', 'region']], on='customer_id', how='left')
            # Step 4: Add derived columns
            .assign(
                transaction_month=lambda x: x['date'].dt.to_period('M'),
                price_segment=lambda x: pd.cut(
                    x['total_amount'],
                    bins=[0, 50, 100, 500, float('inf')],
                    labels=['Budget', 'Standard', 'Premium', 'Luxury']
                )
            )
            # Step 5: Sort by date
            .sort_values('date')
            # Step 6: Reset index
            .reset_index(drop=True)
        )
        
        print(f"\nPipeline output:")
        print(f"  Shape: {result.shape}")
        print(f"  Columns: {len(result.columns)}")
        print(f"\nSample result:")
        print(result[['customer_id', 'category', 'region', 'total_amount', 'price_segment']].head(10))
        
        return result


# Execute Task 4
task4 = Task4_CustomExtensions(transactions_df, products_df, customers_df)
sales_df = task4.custom_accessors_demo()
pipeline_result = task4.method_chaining_pipeline()

print("\n✓ Task 4 Completed Successfully!")


████████████████████████████████████████████████████████████████████████████████
█ TASK 4: CUSTOM EXTENSIONS & ADVANCED FUNCTIONALITY
████████████████████████████████████████████████████████████████████████████████

4.1: CUSTOM ACCESSORS

1. Sales Accessor Methods
----------------------------------------
Total Revenue: $36,123,666.17
Avg Transaction: $1191.41

Revenue by Category:
  Food           : $10,464,437.49 ( 29.0%)
  Electronics    : $8,412,826.42 ( 23.3%)
  Beauty         : $7,414,828.17 ( 20.5%)
  Home           : $6,179,807.47 ( 17.1%)
  Clothing       : $3,651,766.62 ( 10.1%)

Discount Analysis:
  % Transactions Discounted: 20.0%
  Average Discount: 12.44%
  Revenue (Discounted): $6,530,673.25

2. Quality Accessor Methods
----------------------------------------
Data Completeness: 99.8%
Duplicate Ratio: 0.49%

Missing Values by Column:
  unit_price: 1.00%
  discount_pct: 0.99%

4.2: METHOD CHAINING PIPELINES

Building data processing pipeline...
---------------------------

## 6. TASK 5: Real-World Challenge - Retail Analytics

### Topics Covered:
1. **Data Cleaning** - Handle missing values, duplicates, outliers
2. **Customer Profiling** - Create RFM segments
3. **Product Analytics** - Revenue analysis, recommendations
4. **Anomaly Detection** - Statistical and behavioral anomalies
5. **Business Insights** - Dashboard generation

### Best Practices:
- Always validate data quality before analysis
- Document business rules clearly
- Create reproducible analysis pipelines
- Validate results with domain experts

In [67]:
# ============================================================================
# TASK 5: REAL-WORLD CHALLENGE - RETAIL ANALYTICS
# ============================================================================

print("\n" + "█"*80)
print("█ TASK 5: REAL-WORLD CHALLENGE - RETAIL ANALYTICS")
print("█"*80)

class Task5_RealWorldChallenge:
    """
    Real-world retail analytics challenge demonstrating end-to-end analysis.
    
    Methods:
    - data_cleaning: Handle data quality issues
    - customer_profiling: Create customer segments
    - product_analysis: Revenue and performance analysis
    - generate_insights: Create actionable insights
    """
    
    def __init__(self, transactions_df, products_df, customers_df, status_updates_df):
        """Initialize with all data sources"""
        self.transactions = transactions_df.copy()
        self.products = products_df.copy()
        self.customers = customers_df.copy()
        self.status_updates = status_updates_df.copy()
    
    def data_cleaning(self):
        """
        Clean data by handling:
        - Missing values
        - Duplicate records
        - Outliers and anomalies
        """
        print("\n" + "="*80)
        print("5.1: DATA CLEANING")
        print("="*80)
        
        df = self.transactions.copy()
        
        # ===== MISSING VALUES ANALYSIS =====
        print("\n1. Missing Values Analysis")
        print("-" * 40)
        
        missing_counts = df.isnull().sum()
        missing_pct = (missing_counts / len(df)) * 100
        missing_info = pd.DataFrame({
            'Missing_Count': missing_counts,
            'Missing_Percent': missing_pct
        })
        
        print(missing_info[missing_info['Missing_Count'] > 0])
        
        # Handle missing values
        df_cleaned = df.copy()
        
        # Fill numeric missing values with mean
        numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if df_cleaned[col].isnull().any():
                df_cleaned[col].fillna(df_cleaned[col].mean(), inplace=True)
        
        # ===== DUPLICATE HANDLING =====
        print("\n2. Duplicate Record Handling")
        print("-" * 40)
        
        print(f"Duplicates before cleaning: {df_cleaned.duplicated().sum()}")
        df_cleaned = df_cleaned.drop_duplicates(subset=['transaction_id'], keep='first')
        print(f"Duplicates after cleaning: {df_cleaned.duplicated().sum()}")
        
        # ===== OUTLIER DETECTION =====
        print("\n3. Outlier Detection (IQR Method)")
        print("-" * 40)
        
        Q1 = df_cleaned['total_amount'].quantile(0.25)
        Q3 = df_cleaned['total_amount'].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df_cleaned[
            (df_cleaned['total_amount'] < lower_bound) |
            (df_cleaned['total_amount'] > upper_bound)
        ]
        
        print(f"Outliers detected: {len(outliers)} ({len(outliers)/len(df_cleaned)*100:.2f}%)")
        print(f"Bounds: [{lower_bound:.2f}, {upper_bound:.2f}]")
        
        # Remove invalid transactions (negative amounts)
        df_cleaned = df_cleaned[df_cleaned['total_amount'] > 0]
        
        print(f"\nFinal cleaned data: {len(df_cleaned)} records")
        
        return df_cleaned
    
    def customer_profiling(self, clean_transactions):
        """
        Create comprehensive customer profiles and segments.
        
        Metrics:
        - Recency: Days since last purchase
        - Frequency: Number of transactions
        - Monetary: Total spending
        """
        print("\n" + "="*80)
        print("5.2: CUSTOMER PROFILING & SEGMENTATION")
        print("="*80)
        
        # Create RFM metrics
        print("\n1. RFM (Recency, Frequency, Monetary) Analysis")
        print("-" * 40)
        
        customer_metrics = clean_transactions.groupby('customer_id').agg({
            'transaction_id': 'count',     # Frequency
            'total_amount': 'sum',          # Monetary
            'date': 'max'                  # Recency (latest purchase)
        }).reset_index()
        
        customer_metrics.columns = ['customer_id', 'frequency', 'monetary', 'last_purchase']
        
        # Calculate recency (days since last purchase)
        customer_metrics['recency'] = (
            pd.Timestamp.now() - customer_metrics['last_purchase']
        ).dt.days
        
        print(f"\nCustomer Metrics Summary:")
        print(customer_metrics[['frequency', 'monetary', 'recency']].describe())
        
        # ===== CUSTOMER SEGMENTATION =====
        print("\n2. Customer Segmentation")
        print("-" * 40)
        
        # Create segments based on RFM quartiles
        customer_metrics['segment'] = 'Standard'
        
        # VIP: High frequency, high spending, low recency
        vip_cond = (
            (customer_metrics['frequency'] > customer_metrics['frequency'].quantile(0.75)) &
            (customer_metrics['monetary'] > customer_metrics['monetary'].quantile(0.75))
        )
        customer_metrics.loc[vip_cond, 'segment'] = 'VIP'
        
        # At-Risk: Low frequency or high recency
        risk_cond = customer_metrics['recency'] > customer_metrics['recency'].quantile(0.75)
        customer_metrics.loc[risk_cond, 'segment'] = 'At-Risk'
        
        # New: Low frequency but recent
        new_cond = (
            (customer_metrics['frequency'] < customer_metrics['frequency'].quantile(0.25)) &
            (customer_metrics['recency'] < 30)
        )
        customer_metrics.loc[new_cond, 'segment'] = 'New'
        
        print("\nSegment Distribution:")
        segment_stats = customer_metrics.groupby('segment').agg({
            'customer_id': 'count',
            'monetary': ['mean', 'sum'],
            'frequency': 'mean'
        }).round(2)
        
        segment_stats.columns = ['Count', 'Avg_Value', 'Total_Value', 'Avg_Frequency']
        print(segment_stats)
        
        return customer_metrics
    
    def product_analysis(self, clean_transactions):
        """
        Analyze product performance and identify recommendations.
        """
        print("\n" + "="*80)
        print("5.3: PRODUCT ANALYSIS & RECOMMENDATIONS")
        print("="*80)
        
        # Merge with product data
        df = clean_transactions.merge(
            self.products[['product_id', 'category', 'price']],
            on='product_id',
            how='left'
        )
        
        # ===== REVENUE ANALYSIS =====
        print("\n1. Revenue Analysis by Category")
        print("-" * 40)
        
        category_revenue = df.groupby('category').agg({
            'total_amount': ['sum', 'mean', 'count'],
            'quantity': 'mean'
        }).round(2)
        
        category_revenue.columns = ['Total_Revenue', 'Avg_Transaction', 'Num_Transactions', 'Avg_Quantity']
        category_revenue = category_revenue.sort_values('Total_Revenue', ascending=False)
        
        print("\nTop Categories by Revenue:")
        print(category_revenue)
        
        # ===== CO-PURCHASE ANALYSIS =====
        print("\n2. Co-Purchase Patterns")
        print("-" * 40)
        
        # Find products bought together
        customer_products = df.groupby('customer_id')['product_id'].apply(list).reset_index()
        
        copurchase_patterns = {}
        for products in customer_products['product_id']:
            if len(products) > 1:
                for pair in combinations(set(products), 2):
                    key = tuple(sorted(pair))
                    copurchase_patterns[key] = copurchase_patterns.get(key, 0) + 1
        
        # Get top patterns
        top_patterns = sorted(copurchase_patterns.items(), key=lambda x: x[1], reverse=True)[:10]
        print(f"\nFound {len(copurchase_patterns)} unique co-purchase pairs")
        print(f"Top 10 patterns: {len(top_patterns)} pairs")
        
        return category_revenue, copurchase_patterns
    
    def generate_insights(self, clean_transactions):
        """
        Generate business insights and summary statistics.
        """
        print("\n" + "="*80)
        print("5.4: BUSINESS INSIGHTS & SUMMARY")
        print("="*80)
        
        # Merge all data
        df = clean_transactions.merge(
            self.products[['product_id', 'category']],
            on='product_id',
            how='left'
        ).merge(
            self.customers[['customer_id', 'region']],
            on='customer_id',
            how='left'
        )
        
        # ===== KEY METRICS =====
        print("\n1. Key Performance Indicators (KPIs)")
        print("-" * 40)
        
        kpis = {
            'Total Revenue': df['total_amount'].sum(),
            'Number of Transactions': len(df),
            'Average Order Value': df['total_amount'].mean(),
            'Number of Customers': df['customer_id'].nunique(),
            'Number of Products': df['product_id'].nunique(),
            'Revenue per Customer': df['total_amount'].sum() / df['customer_id'].nunique(),
            'Avg Discount': df['discount_pct'].mean() * 100
        }
        
        for metric, value in kpis.items():
            if 'Revenue' in metric or 'Value' in metric:
                print(f"  {metric:30s}: ${value:12,.2f}")
            elif 'Discount' in metric:
                print(f"  {metric:30s}: {value:12.2f}%")
            else:
                print(f"  {metric:30s}: {value:12,.0f}")
        
        # ===== GEOGRAPHIC ANALYSIS =====
        print("\n2. Geographic Performance")
        print("-" * 40)
        
        regional_revenue = df.groupby('region')['total_amount'].sum().sort_values(ascending=False)
        print("\nRevenue by Region:")
        for region, revenue in regional_revenue.items():
            pct = (revenue / df['total_amount'].sum()) * 100
            print(f"  {region:20s}: ${revenue:12,.2f} ({pct:5.1f}%)")
        
        # ===== CATEGORY ANALYSIS =====
        print("\n3. Category Performance")
        print("-" * 40)
        
        category_revenue = df.groupby('category')['total_amount'].sum().sort_values(ascending=False)
        print("\nRevenue by Category:")
        for category, revenue in category_revenue.items():
            pct = (revenue / df['total_amount'].sum()) * 100
            print(f"  {category:20s}: ${revenue:12,.2f} ({pct:5.1f}%)")
        
        return kpis


# Execute Task 5
task5 = Task5_RealWorldChallenge(transactions_df, products_df, customers_df, status_updates_df)
clean_trans = task5.data_cleaning()
customer_profiles = task5.customer_profiling(clean_trans)
category_revenue, copurchase = task5.product_analysis(clean_trans)
kpis = task5.generate_insights(clean_trans)

print("\n✓ Task 5 Completed Successfully!")


████████████████████████████████████████████████████████████████████████████████
█ TASK 5: REAL-WORLD CHALLENGE - RETAIL ANALYTICS
████████████████████████████████████████████████████████████████████████████████

5.1: DATA CLEANING

1. Missing Values Analysis
----------------------------------------
              Missing_Count  Missing_Percent
unit_price              304             1.00
discount_pct            300             0.99

2. Duplicate Record Handling
----------------------------------------
Duplicates before cleaning: 150
Duplicates after cleaning: 0

3. Outlier Detection (IQR Method)
----------------------------------------
Outliers detected: 292 (0.97%)
Bounds: [-1447.26, 3659.22]

Final cleaned data: 30020 records

5.2: CUSTOMER PROFILING & SEGMENTATION

1. RFM (Recency, Frequency, Monetary) Analysis
----------------------------------------

Customer Metrics Summary:
       frequency  monetary  recency
count     500.00    500.00   500.00
mean       60.04  71934.16   829.

## 7. Execution Summary & Recommendations

### Tasks Completed ✓

1. **Task 1: Advanced Data Transformation** ✓
   - Pivot/Melt operations for format conversion
   - Multi-level hierarchical indexing
   - Advanced GroupBy with transform, agg, and custom functions

2. **Task 2: Advanced Merging & Joining** ✓
   - Three-way joins and self-joins
   - Duplicate handling and conflict resolution
   - Time-based joins (AsOf and time-window)

3. **Task 3: Performance Optimization** ✓
   - Memory usage optimization (dtype conversion, categoricals)
   - Computational efficiency (vectorization vs loops)
   - Chunking strategy for large datasets

4. **Task 4: Custom Extensions** ✓
   - Custom pandas accessors for domain-specific operations
   - Method chaining for readable pipelines
   - Advanced functionality demonstrations

5. **Task 5: Real-World Challenge** ✓
   - Data cleaning (missing values, duplicates, outliers)
   - Customer profiling and RFM segmentation
   - Product analysis and recommendations
   - Business insights generation

### Pandas Best Practices Summary

#### ✓ DO:
- Use vectorized operations (NumPy/Pandas) over loops
- Use `.loc[]` for label-based indexing
- Use `.groupby().agg()` for flexible aggregations
- Use `.merge()` with appropriate how parameter
- Use `.astype()` for dtype conversion
- Copy dataframes when needed: `.copy()`

#### ✗ DON'T:
- Never use `.iterrows()` on large datasets
- Avoid `.apply(axis=1)` for simple operations
- Don't modify during iteration
- Avoid creating temporary lists for aggregation
- Don't ignore data quality issues

### Performance Tips

1. **Memory**: Reduce by 30-50% with dtype optimization
2. **Speed**: Vectorization is 10-100x faster than iterrows
3. **Scalability**: Use chunking for datasets > 100MB
4. **Quality**: Always validate merges and duplicates

---

**Created**: January 27, 2026  
**Python Version**: 3.9+  
**Pandas Version**: 1.3.0+

In [68]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*80)
print("ADVANCED PANDAS NOTEBOOK - EXECUTION COMPLETE")
print("="*80)

print("""
✓ TASKS COMPLETED:
  1. Advanced Data Transformation & Reshaping
  2. Advanced Merging & Joining
  3. Performance Optimization & Memory Management
  4. Custom Extensions & Advanced Functionality
  5. Real-World Challenge - Retail Analytics

✓ KEY LEARNINGS:
  • Pivot/Melt for flexible data reshaping
  • Hierarchical indexing for complex data organization
  • Advanced groupby operations with transform/agg
  • Complex merges with duplicate/conflict handling
  • Time-based joins with AsOf and time-windows
  • Memory optimization through dtype management
  • Performance gains through vectorization
  • Custom pandas accessors for domain operations
  • Method chaining for readable pipelines
  • Real-world analytics workflow

✓ BEST PRACTICES:
  • Always vectorize - Never use iterrows
  • Validate data quality before analysis
  • Use method chaining for readability
  • Document all custom operations
  • Optimize memory for large datasets
  • Test merge operations for data integrity

📊 RESOURCES FOR FURTHER LEARNING:
  • https://pandas.pydata.org/docs/
  • https://pandas.pydata.org/docs/user_guide/
  • Real dataset exploration and practice
""")

print("="*80)
print("Thank you for using this comprehensive Pandas guide!")
print("="*80)


ADVANCED PANDAS NOTEBOOK - EXECUTION COMPLETE

✓ TASKS COMPLETED:
  1. Advanced Data Transformation & Reshaping
  2. Advanced Merging & Joining
  3. Performance Optimization & Memory Management
  4. Custom Extensions & Advanced Functionality
  5. Real-World Challenge - Retail Analytics

✓ KEY LEARNINGS:
  • Pivot/Melt for flexible data reshaping
  • Hierarchical indexing for complex data organization
  • Advanced groupby operations with transform/agg
  • Complex merges with duplicate/conflict handling
  • Time-based joins with AsOf and time-windows
  • Memory optimization through dtype management
  • Performance gains through vectorization
  • Custom pandas accessors for domain operations
  • Method chaining for readable pipelines
  • Real-world analytics workflow

✓ BEST PRACTICES:
  • Always vectorize - Never use iterrows
  • Validate data quality before analysis
  • Use method chaining for readability
  • Document all custom operations
  • Optimize memory for large datasets
  • Tes